Este va a ser el archivo donde llevare acabo el proceso de ETL

In [1]:
import pandas as pd
import ast  
import numpy as np

# Cargamos el dataset
df = pd.read_csv("C:/Users/JuanPablo/Desktop/Kazan/Henry/PI_1/credits.csv") 
df

C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\1874185965.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


Como podrán ver contiene columnas anidadas, por lo cuál voy a desanidar

In [2]:
# Convertimos la columna "cast" de string a lista de diccionarios
df["cast"] = df["cast"].apply(ast.literal_eval)

# Desanidamos "cast"
df_exploded = df.explode("cast").reset_index(drop=True)

# Convertimos los diccionarios en columnas separadas y agregamos un prefijo para evitar conflicto de nombres
df_exploded.rename(columns={"id": "movie_id", "id_": "actor_id"}, inplace=True)
df_cast = pd.json_normalize(df_exploded["cast"]).add_prefix("cast_")

# Concatenamos con el DataFrame original sin duplicar la columna "cast"
cast = pd.concat([df_exploded.drop(columns=["cast"]), df_cast], axis=1)
cast.drop(columns="crew", inplace=True)
cast.rename(columns={"cast_id":"actor_id","cast_cast_id" : "cast_id"}, inplace=True)
cast

,movie_id,cast_id,cast_character,cast_credit_id,cast_gender,actor_id,cast_name,cast_order,cast_profile_path
0,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,862,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,862,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,862,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...,...
564887,227506,3.0,,52fe4ea59251416c7515d7d9,1.0,1090923.0,Nathalie Lissenko,1.0,None
564888,227506,4.0,,52fe4ea59251416c7515d7dd,2.0,1136422.0,Pavel Pavlov,2.0,None
564889,227506,5.0,,52fe4ea59251416c7515d7e1,0.0,1261758.0,Aleksandr Chabrov,3.0,None
564890,227506,6.0,,52fe4ea59251416c7515d7e5,1.0,29199.0,Vera Orlova,4.0,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg


Dividí los dataframes en "cast" y "crew" ya que si trataba de desanidar ambos juntos, no me dejaba, y aparte separados creo que va a ser más facil de manejar por el momento

Y estos códigos los dividi porque tampoco los podía procesar juntos, pero los 3 son el proceso de desanidado de la columna "crew":

In [3]:
df = pd.read_csv("C:/Users/JuanPablo/Desktop/Kazan/Henry/PI_1/credits.csv") 
df["crew"] = df["crew"].apply(ast.literal_eval)

In [4]:
crew_df = df.explode("crew")

In [5]:
# Convertir los diccionarios en columnas
crew_df = pd.concat([crew_df.drop(columns=["crew"]), crew_df["crew"].apply(pd.Series)], axis=1)

In [6]:
# Mostrar el resultado
print(crew_df.head())

                                                cast   id  \
0  [{'cast_id': 14, 'character': 'Woody (voice)',...  862   
0  [{'cast_id': 14, 'character': 'Woody (voice)',...  862   
0  [{'cast_id': 14, 'character': 'Woody (voice)',...  862   
0  [{'cast_id': 14, 'character': 'Woody (voice)',...  862   
0  [{'cast_id': 14, 'character': 'Woody (voice)',...  862   

                  credit_id department  gender       id         job  \
0  52fe4284c3a36847f8024f49  Directing     2.0   7879.0    Director   
0  52fe4284c3a36847f8024f4f    Writing     2.0  12891.0  Screenplay   
0  52fe4284c3a36847f8024f55    Writing     2.0      7.0  Screenplay   
0  52fe4284c3a36847f8024f5b    Writing     2.0  12892.0  Screenplay   
0  52fe4284c3a36847f8024f61    Writing     0.0  12893.0  Screenplay   

             name                      profile_path   0  
0   John Lasseter  /7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg NaN  
0     Joss Whedon  /dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg NaN  
0  Andrew Stanton  /pvQWsu0qc8JF

Y acá adelante voy a limpiar el df de "crew". En el primer codigo, diferenciamos id de movie_id con el id de crew_id, ya que sino los dos se llaman id

In [7]:
crew_df.columns = [f"{col}_{i}" if crew_df.columns.duplicated()[i] else col 
              for i, col in enumerate(crew_df.columns)]
crew_df

,cast,id,credit_id,department,gender,id_5,job,name,profile_path,0
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,NaN
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg,NaN
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg,NaN
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg,NaN
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg,NaN
...,...,...,...,...,...,...,...,...,...,...
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...",67758,52fe4776c3a368484e0c8399,Sound,0.0,549356.0,Original Music Composer,Richard McHugh,None,NaN
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...",67758,52fe4776c3a368484e0c839f,Camera,2.0,58818.0,Director of Photography,João Fernandes,None,NaN
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...",227506,533bccebc3a36844cf0011a7,Directing,0.0,1085341.0,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg,NaN
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...",227506,58ebbc26925141281908aa0a,Production,2.0,1195656.0,Producer,Joseph N. Ermolieff,None,NaN


Y acá vamos a eliminar la última columna, que se llama 0 y todos sus valores son Nan, la columna cast, y renombramos las columnas para evitar confuciones 

In [8]:
crew_df = crew_df.iloc[:, :-1]
crew_df = crew_df.drop('cast', axis=1)
crew_df.rename(columns= {"id": "movie_id","credit_id":"crew_credit_id", "department":"crew_department", "gender":"crew_gender", 
                          "id_5":"crew_id", "job":"crew_job", "name":"crew_name", "profile_path": "crew_profile_path"}, inplace=True)
crew_df


,movie_id,crew_credit_id,crew_department,crew_gender,crew_id,crew_job,crew_name,crew_profile_path
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
0,862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
0,862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
0,862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
0,862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
...,...,...,...,...,...,...,...,...
45473,67758,52fe4776c3a368484e0c8399,Sound,0.0,549356.0,Original Music Composer,Richard McHugh,None
45473,67758,52fe4776c3a368484e0c839f,Camera,2.0,58818.0,Director of Photography,João Fernandes,None
45474,227506,533bccebc3a36844cf0011a7,Directing,0.0,1085341.0,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg
45474,227506,58ebbc26925141281908aa0a,Production,2.0,1195656.0,Producer,Joseph N. Ermolieff,None


Ahora vamos con el csv de "movies"

In [9]:
m = pd.read_csv("C:/Users/JuanPablo/Desktop/Kazan/Henry/PI_1/movies_dataset.csv", dtype=str)
print(m.head(15))

    adult                              belongs_to_collection    budget  \
0   False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1   False                                                NaN  65000000   
2   False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3   False                                                NaN  16000000   
4   False  {'id': 96871, 'name': 'Father of the Bride Col...         0   
5   False                                                NaN  60000000   
6   False                                                NaN  58000000   
7   False                                                NaN         0   
8   False                                                NaN  35000000   
9   False  {'id': 645, 'name': 'James Bond Collection', '...  58000000   
10  False                                                NaN  62000000   
11  False                                                NaN         0   
12  False  {'id': 117693, 'name': 'Bal

Nos deshacemos de las columnas que no vamos a necesitar en el proyecto

In [10]:
m.drop(columns="adult", inplace=True)
m.drop(columns="belongs_to_collection", inplace=True)
m.drop(columns="genres", inplace=True)
m.drop(columns="homepage", inplace=True)
m.drop(columns="imdb_id", inplace=True)
m.drop(columns="original_language", inplace=True)
m.drop(columns="overview", inplace=True)
m.drop(columns="spoken_languages", inplace=True)
m.drop(columns="status", inplace=True)
m.drop(columns="tagline", inplace=True)
m.drop(columns="original_title", inplace=True)
m.drop(columns="video", inplace=True)
m.drop(columns="poster_path", inplace=True)
m.drop(columns="production_companies", inplace=True)
m.drop(columns="production_countries", inplace=True)
m

,budget,id,popularity,release_date,revenue,runtime,title,vote_average,vote_count
0,30000000,862,21.946943,1995-10-30,373554033,81.0,Toy Story,7.7,5415
1,65000000,8844,17.015539,1995-12-15,262797249,104.0,Jumanji,6.9,2413
2,0,15602,11.7129,1995-12-22,0,101.0,Grumpier Old Men,6.5,92
3,16000000,31357,3.859495,1995-12-22,81452156,127.0,Waiting to Exhale,6.1,34
4,0,11862,8.387519,1995-02-10,76578911,106.0,Father of the Bride Part II,5.7,173
...,...,...,...,...,...,...,...,...,...
45461,0,439050,0.072051,NaN,0,90.0,Subdue,4.0,1
45462,0,111109,0.178241,2011-11-17,0,360.0,Century of Birthing,9.0,3
45463,0,67758,0.903007,2003-08-01,0,90.0,Betrayal,3.8,6
45464,0,227506,0.003503,1917-10-21,0,87.0,Satan Triumphant,0.0,0


Para mayor manejo, vamos a traer solamente las primeras 2.000 filas de cada df

In [ ]:
cast = cast.head(2000)
crew = crew_df.head(2000)
movies = m.head(2000)

Los valores nulos de los campos revenue y budget deben ser rellenos por el número 0.

In [13]:
# Buscamos las columnas
print(crew.columns)
print(cast.columns)
print(movies.columns)

Index(['movie_id', 'crew_credit_id', 'crew_department', 'crew_gender',
       'crew_id', 'crew_job', 'crew_name', 'crew_profile_path'],
      dtype='object')
Index(['movie_id', 'cast_id', 'cast_character', 'cast_credit_id',
       'cast_gender', 'actor_id', 'cast_name', 'cast_order',
       'cast_profile_path'],
      dtype='object')
Index(['budget', 'id', 'popularity', 'release_date', 'revenue', 'runtime',
       'title', 'vote_average', 'vote_count'],
      dtype='object')


In [14]:
# Detectamos que están en el df de Movies y reemplazamos
movies["revenue"].fillna(0, inplace=True)  
movies["budget"].fillna(0, inplace=True)  
print(movies.isnull().sum())

budget          0
id              0
popularity      0
release_date    2
revenue         0
runtime         5
title           0
vote_average    0
vote_count      0
dtype: int64


C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\2074265551.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies["revenue"].fillna(0, inplace=True)
C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\2074265551.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["revenue"].fillna(0, inplace=True)
C:\Users\JuanPablo\AppData\Lo

Eliminamos los nulos de "release_date"

In [15]:
movies.dropna(subset="release_date", inplace=True)
print(m.isnull().sum())

budget            0
id                0
popularity        5
release_date     87
revenue           6
runtime         263
title             6
vote_average      6
vote_count        6
dtype: int64


C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\1201454224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.dropna(subset="release_date", inplace=True)


De haber fechas, deberá tener el formato AAAA-mm-dd, además deberá crear la columna release_yeardonde extraerán el año de la fecha de estreno.

In [16]:
# Revisamos
movies

,budget,id,popularity,release_date,revenue,runtime,title,vote_average,vote_count
0,30000000,862,21.946943,1995-10-30,373554033,81.0,Toy Story,7.7,5415
1,65000000,8844,17.015539,1995-12-15,262797249,104.0,Jumanji,6.9,2413
2,0,15602,11.7129,1995-12-22,0,101.0,Grumpier Old Men,6.5,92
3,16000000,31357,3.859495,1995-12-22,81452156,127.0,Waiting to Exhale,6.1,34
4,0,11862,8.387519,1995-02-10,76578911,106.0,Father of the Bride Part II,5.7,173
...,...,...,...,...,...,...,...,...,...
1995,0,27332,5.540117,1982-07-30,0,103.0,Tex,5.9,9
1996,17000000,97,16.574021,1982-07-09,33000000,96.0,Tron,6.6,717
1997,0,10419,4.379054,1993-03-05,5632086,112.0,Swing Kids,6.7,63
1998,17000000,11675,8.619068,1998-08-05,55041738,86.0,Halloween: H20,5.7,306


Por lo que vemos, se encuentra en formato AAAA-MM-DD

In [17]:
print(movies['release_date'].dtype)
# La columna release_date no esta en formato fecha, la convertimos
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
print(movies['release_date'].dtype)
movies['release_year'] = movies['release_date'].dt.year
print(movies["release_year"])

object
datetime64[ns]
0       1995
1       1995
2       1995
3       1995
4       1995
        ... 
1995    1982
1996    1982
1997    1993
1998    1998
1999    1991
Name: release_year, Length: 1998, dtype: int32


C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\713851295.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\713851295.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['release_year'] = movies['release_date'].dt.year


Crear la columna con el retorno de inversión, llamada returncon los campos revenuey budget, dividiendo estas dos últimas revenue / budget, cuando no haya datos disponibles para calcularlo, deberá tomar el valor 0

In [18]:
print(movies["budget"].dtype)
print(movies["revenue"].dtype)
# Necesitamos pasar ambas a valores numericos para poder trabajar con Numpy
movies['budget'] = pd.to_numeric(movies['budget'], errors="coerce")
movies['revenue'] = pd.to_numeric(movies['revenue'], errors="coerce")
print(movies["budget"].dtype)
print(movies["revenue"].dtype)
movies["budget"].fillna(0, inplace=True) # Reemplazamos los NaN por 0s
movies["revenue"].fillna(0, inplace=True)

movies["returncon"] = np.where(
    (movies["budget"]==0) | (movies["revenue"]==0), # Para que no haya división por 0, que darían infinito
     0,
    movies["revenue"]/movies["budget"])

print(movies["returncon"])

object
object
int64
int64
0       12.451801
1        4.043035
2        0.000000
3        5.090760
4        0.000000
          ...    
1995     0.000000
1996     1.941176
1997     0.000000
1998     3.237749
1999     0.000000
Name: returncon, Length: 1998, dtype: float64


C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\1926734646.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['budget'] = pd.to_numeric(movies['budget'], errors="coerce")
C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\1926734646.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['revenue'] = pd.to_numeric(movies['revenue'], errors="coerce")
C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\1926734646.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or S

Eliminar las columnas que no serán utilizadas, video, imdb_id, adult, original_title, poster_pathy homepage.

In [19]:
# Buscamos las columnas
print(crew.columns)
print(cast.columns)
print(movies.columns)

Index(['movie_id', 'crew_credit_id', 'crew_department', 'crew_gender',
       'crew_id', 'crew_job', 'crew_name', 'crew_profile_path'],
      dtype='object')
Index(['movie_id', 'cast_id', 'cast_character', 'cast_credit_id',
       'cast_gender', 'actor_id', 'cast_name', 'cast_order',
       'cast_profile_path'],
      dtype='object')
Index(['budget', 'id', 'popularity', 'release_date', 'revenue', 'runtime',
       'title', 'vote_average', 'vote_count', 'release_year', 'returncon'],
      dtype='object')


Ya están eliminadas

In [20]:
crew.drop(columns="crew_gender", inplace=True)
crew.drop(columns="crew_credit_id", inplace=True)
crew.drop(columns="crew_profile_path", inplace=True)
crew

C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\3890621395.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crew.drop(columns="crew_gender", inplace=True)
C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\3890621395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crew.drop(columns="crew_credit_id", inplace=True)
C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\3890621395.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

,movie_id,crew_department,crew_id,crew_job,crew_name
0,862,Directing,7879.0,Director,John Lasseter
0,862,Writing,12891.0,Screenplay,Joss Whedon
0,862,Writing,7.0,Screenplay,Andrew Stanton
0,862,Writing,12892.0,Screenplay,Joel Cohen
0,862,Writing,12893.0,Screenplay,Alec Sokolow
...,...,...,...,...,...
84,8447,Production,55039.0,Producer,Belinda Haas
84,8447,Production,11381.0,Producer,Joyce Herlihy
84,8447,Production,38082.0,Producer,Kerry Orent
84,8447,Editing,55039.0,Editor,Belinda Haas


In [21]:
cast.drop(columns="cast_gender", inplace=True)
cast.drop(columns="cast_credit_id", inplace=True)
cast.drop(columns="cast_profile_path", inplace=True)
cast.drop(columns="cast_order", inplace=True)
cast

C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\3583197100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cast.drop(columns="cast_gender", inplace=True)
C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\3583197100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cast.drop(columns="cast_credit_id", inplace=True)
C:\Users\JuanPablo\AppData\Local\Temp\ipykernel_1488\3583197100.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

,movie_id,cast_id,cast_character,actor_id,cast_name
0,862,14.0,Woody (voice),31.0,Tom Hanks
1,862,15.0,Buzz Lightyear (voice),12898.0,Tim Allen
2,862,16.0,Mr. Potato Head (voice),7167.0,Don Rickles
3,862,17.0,Slinky Dog (voice),12899.0,Jim Varney
4,862,18.0,Rex (voice),12900.0,Wallace Shawn
...,...,...,...,...,...
1995,11066,1.0,Marcus Graham,776.0,Eddie Murphy
1996,11066,2.0,Jacqueline 'Jackie / Jack' Broyer,59153.0,Robin Givens
1997,11066,3.0,Angela Lewis,4587.0,Halle Berry
1998,11066,4.0,Gerard Jackson,58563.0,David Alan Grier


Ya tenemos los datos listos para exportar

In [22]:
movies.to_csv("C:/Users/JuanPablo/Desktop/Kazan/Henry/PI_1/movies.csv")
cast.to_csv("C:/Users/JuanPablo/Desktop/Kazan/Henry/PI_1/cast.csv")
crew.to_csv("C:/Users/JuanPablo/Desktop/Kazan/Henry/PI_1/crew.csv")